In [1]:
import gc
import glob
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from random import randrange
from tqdm import tqdm, tnrange
from tqdm.notebook import tqdm_notebook

import warnings

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from keras.optimizer_v2.gradient_descent import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Lambda, Dropout, LeakyReLU, BatchNormalization, Activation, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

from mlxtend.evaluate import accuracy

from tensorflow.keras.optimizers import Adam,RMSprop,SGD, Nadam
#from tensorflow.keras.utils import to_categorical
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.applications import VGG16

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # ignore tensorflow warnings
warnings.filterwarnings("ignore")

In [3]:
path_data = "data/data_palm_vein/NIR"

In [4]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=1007,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)


Found 6000 files belonging to 500 classes.
Using 4800 files for training.
Metal device set to: Apple M1


In [5]:
len(train.class_names)

500

In [6]:
from keras.applications.vgg19 import VGG19

def vgg_model_tf(input_shape=(128, 128, 3), nombre_classes=500):
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    vgg19.trainable = False
    model = Sequential()
    model.add(vgg19)
    model.add(Flatten())
    model.add(Dense(4096, activation='LeakyReLU'))
    model.add(Dense(4096, activation='LeakyReLU'))
    model.add(Dense(nombre_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    model.summary()
    return model

In [7]:
model = vgg_model_tf()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 4096)              33558528  
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 500)               2048500   
Total params: 72,412,724
Trainable params: 52,388,340
Non-trainable params: 20,024,384
_________________________________________________________________


In [8]:
epochs = 10
batch = 50
print("\n ================= Training : VGG16 model ================= \n")
print("             Epochs :  {}   |   Batch size : {} ".format(epochs, batch))
print("\n ========================================================== \n")
trained = model.fit(train, validation_data = train,epochs=epochs, batch_size=batch)


 ================= Training : VGG16 model ================= 

             Epochs :  10   |   Batch size : 50 


Epoch 1/10
150/150 [==============================] - 106s 709ms/step - loss: 33.0886 - accuracy: 0.3498 - val_loss: 0.4333 - val_accuracy: 0.9352
Epoch 2/10
150/150 [==============================] - 106s 710ms/step - loss: 0.6682 - accuracy: 0.9200 - val_loss: 0.6285 - val_accuracy: 0.9325
Epoch 3/10
150/150 [==============================] - 106s 711ms/step - loss: 455.4421 - accuracy: 0.6996 - val_loss: 679.8967 - val_accuracy: 0.0302
Epoch 4/10
150/150 [==============================] - 106s 711ms/step - loss: 177.9415 - accuracy: 0.5927 - val_loss: 3.5223 - val_accuracy: 0.9354
Epoch 5/10
150/150 [==============================] - 108s 725ms/step - loss: 2.7728 - accuracy: 0.9435 - val_loss: 1.9737 - val_accuracy: 0.9656
Epoch 6/10
150/150 [==============================] - 114s 760ms/step - loss: 1.5510 - accuracy: 0.9721 - val_loss: 1.1947 - val_accuracy: 0.9781
Epo

In [9]:
val = model.evaluate(train)
input_shape = (128, 128, 3)

print("\n ================= Evaluation : VGG16 model ================= \n")
print("  With : \n")
print("Batch size         :  {}     |   Epochs      : {} ".format(batch, epochs))
print("Nombres de classes :  {}    |   Input shape : {} \n".format(len(train.class_names), input_shape))
print("\n ============================================================ \n")

print("  Results : \n")
print("Loss  : %.2f%%" % (val[0] * 100))
print("Score : %.2f%%" % (val[1] * 100))

150/150 [==============================] - 42s 280ms/step - loss: 5.7341 - accuracy: 0.9277

 ================= Evaluation : VGG16 model ================= 

  With : 

Batch size         :  50     |   Epochs      : 10 
Nombres de classes :  500    |   Input shape : (128, 128, 3) 



  Results : 

Loss  : 573.41%
Score : 92.77%
